In [1]:
import os
import sys
import numpy as np
from PIL import Image
import heapq
from google.colab import drive
from io import BytesIO
import base64
from ipywidgets import Button, Dropdown, Image as IPyImage, VBox, Output, Label
from IPython.display import display

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path ke folder gambar
train_folder = '/content/drive/MyDrive/+ Works/datasetbanana/train'
valid_folder = '/content/drive/MyDrive/+ Works/datasetbanana/valid'
test_folder = '/content/drive/MyDrive/+ Works/datasetbanana/test'

In [4]:

# Fungsi untuk memuat gambar dari folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            images.append(filename)
    return images

# Memuat daftar nama file gambar dari folder
image_files = load_images_from_folder(train_folder)

In [5]:
# Implementasi Huffman Coding
class HuffmanNode:
    def __init__(self, pixel=None, freq=0):
        self.left = None
        self.right = None
        self.pixel = pixel
        self.freq = freq

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(freq_dict):
    heap = [HuffmanNode(pixel, freq) for pixel, freq in freq_dict.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(freq=left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)
    return heap[0]

def build_huffman_codes(node, prefix="", codes={}):
    if node is not None:
        if node.pixel is not None:
            codes[node.pixel] = prefix
        build_huffman_codes(node.left, prefix + "0", codes)
        build_huffman_codes(node.right, prefix + "1", codes)
    return codes

def compress_image(image, codes):
    compressed_data = ""
    pixel_values = np.array(image).flatten()
    for pixel in pixel_values:
        compressed_data += codes[pixel]
    return compressed_data

def decompress_image(compressed_data, root, shape):
    current_node = root
    decompressed_pixels = []
    for bit in compressed_data:
        if bit == "0":
            current_node = current_node.left
        else:
            current_node = current_node.right
        if current_node.pixel is not None:
            decompressed_pixels.append(current_node.pixel)
            current_node = root
    return np.array(decompressed_pixels).reshape(shape)

def calculate_pixel_freq(image):
    pixel_freq = {}
    pixels = np.array(image).flatten()
    for pixel in pixels:
        if pixel in pixel_freq:
            pixel_freq[pixel] += 1
        else:
            pixel_freq[pixel] = 1
    return pixel_freq

def image_to_byte_array(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    return img_byte_arr.getvalue()

In [6]:
# Fungsi untuk memilih dan menampilkan gambar serta hasil kompresi
def select_image(change):
    img_path = os.path.join(train_folder, change['new'])
    original_img = Image.open(img_path).convert('L')  # Convert to grayscale
    original_img.thumbnail((256, 256))  # Resize for display purposes
    pixel_freq = calculate_pixel_freq(original_img)
    huffman_tree = build_huffman_tree(pixel_freq)
    huffman_codes = build_huffman_codes(huffman_tree)
    compressed_data = compress_image(original_img, huffman_codes)
    decompressed_pixels = decompress_image(compressed_data, huffman_tree, original_img.size)
    decompressed_img = Image.fromarray(decompressed_pixels)

    original_img_bytes = image_to_byte_array(original_img)
    decompressed_img_bytes = image_to_byte_array(decompressed_img)

    original_size = len(original_img_bytes)
    compressed_size = len(compressed_data) // 32  # Convert bit length to byte length

    with output:
        output.clear_output()
        display(VBox([
            Label('Original Image:'),
            IPyImage(value=original_img_bytes, format='png', width=256, height=256),
            Label('Decompressed Image:'),
            IPyImage(value=decompressed_img_bytes, format='png', width=256, height=256),
            Label(f'Original Size: {original_size} bytes'),
            Label(f'Compressed Size: {compressed_size} bytes'),
            Label(f'Compression Ratio: {(original_size - compressed_size) / original_size:.2f}')
        ]))

# Dropdown untuk memilih gambar
dropdown = Dropdown(
    options=image_files,
    description='Select Image:',
    disabled=False,
)

# Output widget untuk menampilkan gambar
output = Output()

# Observe dropdown changes
dropdown.observe(select_image, names='value')

# Display widgets
display(VBox([dropdown, output]))

## Wavelet compression

In [7]:
!pip install pywavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.8 MB/s eta 0:00:00


In [8]:
import os
import pywt
import numpy as np
from PIL import Image
from io import BytesIO
from ipywidgets import Dropdown, Output, VBox, Label, Image as IPyImage
from IPython.display import display

# Path ke folder gambar
train_folder = '/content/drive/MyDrive/+ Works/datasetbanana/train'
valid_folder = '/content/drive/MyDrive/+ Works/datasetbanana/valid'
test_folder = '/content/drive/MyDrive/+ Works/datasetbanana/test'

# Fungsi untuk memuat gambar dari folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            images.append(filename)
    return images

# Memuat daftar nama file gambar dari folder
image_files = load_images_from_folder(train_folder)

# Implementasi Wavelet Compression
def compress_image_wavelet(image, wavelet='haar', level=1):
    coeffs = pywt.wavedec2(np.array(image), wavelet, level=level)
    coeffs_flat, coeff_slices = pywt.coeffs_to_array(coeffs)
    threshold = np.percentile(np.abs(coeffs_flat), 50)
    coeffs_flat[np.abs(coeffs_flat) < threshold] = 0
    return coeffs_flat, coeff_slices

def decompress_image_wavelet(coeffs_flat, coeff_slices, wavelet='haar'):
    coeffs_from_array = pywt.array_to_coeffs(coeffs_flat, coeff_slices, output_format='wavedec2')
    image_array = pywt.waverec2(coeffs_from_array, wavelet)
    image_array = np.clip(image_array, 0, 255)
    return Image.fromarray(image_array.astype(np.uint8))

def image_to_byte_array(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    return img_byte_arr.getvalue()

# Fungsi untuk memilih dan menampilkan gambar serta hasil kompresi
def select_image(change):
    img_path = os.path.join(train_folder, change['new'])
    original_img = Image.open(img_path).convert('L')  # Convert to grayscale
    original_img.thumbnail((256, 256))  # Resize for display purposes

    coeffs_flat, coeff_slices = compress_image_wavelet(original_img)
    decompressed_img = decompress_image_wavelet(coeffs_flat, coeff_slices)

    original_img_bytes = image_to_byte_array(original_img)
    decompressed_img_bytes = image_to_byte_array(decompressed_img)

    original_size = len(original_img_bytes)
    compressed_size = len(coeffs_flat.tobytes())  // 32# Calculate compressed size in bytes

    with output:
        output.clear_output()
        display(VBox([
            Label('Original Image:'),
            IPyImage(value=original_img_bytes, format='png', width=256, height=256),
            Label('Decompressed Image:'),
            IPyImage(value=decompressed_img_bytes, format='png', width=256, height=256),
            Label(f'Original Size: {original_size} bytes'),
            Label(f'Compressed Size: {compressed_size} bytes'),
            Label(f'Compression Ratio: {(original_size - compressed_size) / original_size:.2f}')
        ]))

# Dropdown untuk memilih gambar
dropdown = Dropdown(
    options=image_files,
    description='Select Image:',
    disabled=False,
)

# Output widget untuk menampilkan gambar
output = Output()

# Observe dropdown changes
dropdown.observe(select_image, names='value')

# Display widgets
display(VBox([dropdown, output]))



## DCT Compression

In [9]:
import os
import numpy as np
from PIL import Image
from google.colab import drive
from io import BytesIO
import scipy.fftpack as fftpack
from ipywidgets import Dropdown, Image as IPyImage, VBox, Output, Label
from IPython.display import display

# Mount Google Drive
drive.mount('/content/drive')

# Path ke folder gambar
train_folder = '/content/drive/MyDrive/+ Works/datasetbanana/train'
valid_folder = '/content/drive/MyDrive/+ Works/datasetbanana/valid'
test_folder = '/content/drive/MyDrive/+ Works/datasetbanana/test'

# Fungsi untuk memuat gambar dari folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            images.append(filename)
    return images

# Memuat daftar nama file gambar dari folder
image_files = load_images_from_folder(valid_folder)

# Fungsi untuk mengkompresi gambar menggunakan DCT
def dct2(block):
    return fftpack.dct(fftpack.dct(block.T, norm='ortho').T, norm='ortho')

def idct2(block):
    return fftpack.idct(fftpack.idct(block.T, norm='ortho').T, norm='ortho')

def dct_compress(image, block_size=8, threshold=0.01):
    h, w = image.shape
    dct_matrix = np.zeros_like(image, dtype=np.float32)

    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = image[i:i+block_size, j:j+block_size]
            dct_block = dct2(block)
            # Apply threshold to truncate coefficients
            dct_block[np.abs(dct_block) < threshold * np.max(dct_block)] = 0
            dct_matrix[i:i+block_size, j:j+block_size] = dct_block

    return dct_matrix, image.shape

def dct_decompress(dct_matrix, original_shape, block_size=8):
    h, w = original_shape
    decompressed_image = np.zeros_like(dct_matrix, dtype=np.float32)

    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = dct_matrix[i:i+block_size, j:j+block_size]
            idct_block = idct2(block)
            decompressed_image[i:i+block_size, j:j+block_size] = idct_block

    decompressed_image = np.clip(decompressed_image, 0, 255)  # Clip to valid pixel range
    decompressed_image = decompressed_image.astype('uint8')
    return decompressed_image

def image_to_byte_array(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    return img_byte_arr.getvalue()

# Fungsi untuk memilih dan menampilkan gambar serta hasil kompresi
def select_image(change):
    img_path = os.path.join(valid_folder, change['new'])
    original_img = Image.open(img_path).convert('L')  # Convert to grayscale
    original_img.thumbnail((256, 256))  # Resize for display purposes
    original_img_np = np.array(original_img)

    dct_matrix, original_shape = dct_compress(original_img_np, block_size=8, threshold=0.01)
    decompressed_img_np = dct_decompress(dct_matrix, original_shape, block_size=8)
    decompressed_img = Image.fromarray(decompressed_img_np)

    original_img_bytes = image_to_byte_array(original_img)
    decompressed_img_bytes = image_to_byte_array(decompressed_img)

    original_size = len(original_img_bytes)
    compressed_size = len(dct_matrix)  # Convert bit length to byte length

    with output:
        output.clear_output()
        display(VBox([
            Label('Original Image:'),
            IPyImage(value=original_img_bytes, format='png', width=256, height=256),
            Label('Decompressed Image:'),
            IPyImage(value=decompressed_img_bytes, format='png', width=256, height=256),
            Label(f'Original Size: {original_size} bytes'),
            Label(f'Compressed Size: {compressed_size} bytes'),
            Label(f'Compression Ratio: {original_size / compressed_size:.2f}')
        ]))

# Dropdown untuk memilih gambar
dropdown = Dropdown(
    options=image_files,
    description='Select Image:',
    disabled=False,
)

# Output widget untuk menampilkan gambar
output = Output()

# Observe dropdown changes
dropdown.observe(select_image, names='value')

# Display widgets
display(VBox([dropdown, output]))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## JPEG Compression

In [10]:
import os
import numpy as np
from PIL import Image
from google.colab import drive
from io import BytesIO
from ipywidgets import Dropdown, Image as IPyImage, VBox, Output, Label
from IPython.display import display

# Mount Google Drive
drive.mount('/content/drive')

# Path ke folder gambar
train_folder = '/content/drive/MyDrive/+ Works/datasetbanana/train'
valid_folder = '/content/drive/MyDrive/+ Works/datasetbanana/valid'
test_folder = '/content/drive/MyDrive/+ Works/datasetbanana/test'

# Fungsi untuk memuat gambar dari folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            images.append(filename)
    return images

# Memuat daftar nama file gambar dari folder
image_files = load_images_from_folder(valid_folder)

# Fungsi untuk mengkompresi gambar menggunakan JPEG
def jpeg_compress(image, quality=20):
    compressed_img_byte_arr = BytesIO()
    image.save(compressed_img_byte_arr, format='JPEG', quality=quality)
    return compressed_img_byte_arr.getvalue()

def jpeg_decompress(compressed_img_byte_arr):
    compressed_img = Image.open(BytesIO(compressed_img_byte_arr))
    return compressed_img

def image_to_byte_array(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    return img_byte_arr.getvalue()

# Fungsi untuk memilih dan menampilkan gambar serta hasil kompresi
def select_image(change):
    img_path = os.path.join(valid_folder, change['new'])
    original_img = Image.open(img_path).convert('RGB')  # Convert to RGB
    original_img.thumbnail((256, 256))  # Resize for display purposes

    compressed_img_byte_arr = jpeg_compress(original_img, quality=20)
    decompressed_img = jpeg_decompress(compressed_img_byte_arr)

    original_img_bytes = image_to_byte_array(original_img)
    decompressed_img_bytes = image_to_byte_array(decompressed_img)

    original_size = len(original_img_bytes)
    compressed_size = len(compressed_img_byte_arr)

    with output:
        output.clear_output()
        display(VBox([
            Label('Original Image:'),
            IPyImage(value=original_img_bytes, format='png', width=256, height=256),
            Label('Decompressed Image:'),
            IPyImage(value=decompressed_img_bytes, format='png', width=256, height=256),
            Label(f'Original Size: {original_size} bytes'),
            Label(f'Compressed Size: {compressed_size} bytes'),
            Label(f'Compression Ratio: {original_size / compressed_size:.2f}')
        ]))

# Dropdown untuk memilih gambar
dropdown = Dropdown(
    options=image_files,
    description='Select Image:',
    disabled=False,
)

# Output widget untuk menampilkan gambar
output = Output()

# Observe dropdown changes
dropdown.observe(select_image, names='value')

# Display widgets
display(VBox([dropdown, output]))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
